In [ ]:
! pip install ultralytics

import cv2
from pathlib import Path
from ultralytics import YOLO
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import json
import time

In [ ]:
def extract_features_from_video(model, video_path, output_dir):
  video_path = Path(video_path)
  label, video_name = video_path.name.split("_")[0], video_path.name.split("_")[1]
  print(f"Processing [{video_name}] with label [{label}]...")
  start_time = time.time()
  frames_predictions = model.track(video_path.as_posix(), imgsz=640, classes=[0], save=True)
  delta_time = time.time() - start_time
  print("Pose estimation completed. Saving results...")
  results_serialized = {
      "name": video_name,
      "label": label,
      "results": [
        json.loads(result.tojson()) for result in frames_predictions
      ]
  }
  output_file = f"{output_dir}/{video_path.name}.json"
  with open(output_file, "w") as f:
    f.write(json.dumps(results_serialized))
  print(f"Results saved to [{output_file}]")

In [ ]:
# Percorso del modello di pose estimation
POSE_ESTIMATOR_PATH = "/content/drive/MyDrive/Colab Notebooks/models/yolov8x-pose-p6.pt"

# Percorso della cartella di input contenente i video da analizzare
VIDEOS_DIR = "/content/drive/MyDrive/Colab Notebooks/videos/new_downloads"

# Percorso della cartella di output per le pose stimate
ANNOTATIONS_DIR = "/content/drive/MyDrive/Colab Notebooks/videos/new_downloads/annotations"

# Numero di video da processare
BATCH_SIZE = 12

In [ ]:
Path(ANNOTATIONS_DIR).mkdir(exist_ok=True)
pose_estimator = YOLO(POSE_ESTIMATOR_PATH)

already_processed_videos = [path.stem for path in Path(ANNOTATIONS_DIR).glob("*.json")]
all_videos = [path.name for path in Path(VIDEOS_DIR).glob("*.mp4")]

videos_to_be_processed = [video for video in all_videos if video not in already_processed_videos]

videos_batch = np.random.choice(videos_to_be_processed, min(BATCH_SIZE, len(videos_to_be_processed)), replace=False)

In [ ]:
print(f"Remaining videos to be processed: {len(videos_to_be_processed)}")
for video in videos_to_be_processed:
  print(f"- {video}")

In [ ]:
print(f"Processing batch of {len(videos_batch)} videos...")
print(videos_batch)
for video in videos_batch:
  video_path = [path for path in Path(VIDEOS_DIR).glob("*.mp4") if video in path.name][0]
  extract_features_from_video(pose_estimator, video_path, ANNOTATIONS_DIR)